In [14]:
import pandas as pd
import numpy as np
import glob
import gc
import keras 
from keras.layers import LSTM, Dense, TimeDistributed, Dropout, MaxPooling2D, Conv2D, Input
from keras.layers import Flatten, BatchNormalization, Activation, Reshape, concatenate
from keras.models import Model
import random
from sklearn.model_selection import KFold
import keras.backend as K
from sklearn.utils import class_weight

In [15]:
gc.enable()

In [16]:
steps = 16
height = 108
width = 32

In [17]:
#change the path to where you save the input numpy arrays
path = 'E:/notes_database/32_frames/'
lstm = np.array(glob.glob(path+'lstm_inputs/*'))
cnn = np.array(glob.glob(path+'cnn_inputs/*'))
y_train = np.array(glob.glob(path+'y_train/*'))
c_lstm = np.array(glob.glob(path+'counter_lstm_inputs/*'))
c_cnn = np.array(glob.glob(path+'counter_cnn_inputs/*'))
c_y_train = np.array(glob.glob(path+'counter_y_train/*'))

In [18]:
#The function analyzes the actual accuracy
def load_test_score(model, cnn_test, lstm_test, y_test):
    prediction = model.predict([cnn_test, lstm_test])
    error = (prediction - y_test) ** 2
    error = (error/y_test.shape[0]).sum()
    wrong = 0
    for x in range(prediction.shape[0]):
        for y in range(prediction.shape[1]):
            if abs(prediction[x][y] - y_test[x][y]) > 0.5:
                wrong += 1
                break
    accuracy = 1 - wrong/(prediction.shape[0])
    return error, accuracy

In [19]:
def inputs_to_model(order, num, start, ending):
    cnn_inputs = np.load(cnn[order[num][0]])
    labels_inputs = np.load(y_train[order[num][0]])
    lstm_inputs = np.load(lstm[order[num][0]])
    
    lstm_inputs = np.append(lstm_inputs, np.load(c_lstm[order[num][0]]), axis=0)
    cnn_inputs = np.append(cnn_inputs, np.load(c_cnn[order[num][0]]), axis=0)
    labels_inputs = np.append(labels_inputs, np.load(c_y_train[order[num][0]]), axis=0)
    
    for file in order[num][start+1:ending]:       
        lstm_inputs = np.append(lstm_inputs, np.load(lstm[file]), axis=0)
        cnn_inputs = np.append(cnn_inputs, np.load(cnn[file]), axis=0)
        labels_inputs = np.append(labels_inputs, np.load(y_train[file]), axis=0)
        lstm_inputs = np.append(lstm_inputs, np.load(c_lstm[file]), axis=0)
        cnn_inputs = np.append(cnn_inputs, np.load(c_cnn[file]), axis=0)
        labels_inputs = np.append(labels_inputs, np.load(c_y_train[file]), axis=0)
    
    lstm_inputs = lstm_inputs.astype(np.float16)
    labels_inputs = labels_inputs.astype(np.float16)
    return lstm_inputs, cnn_inputs, labels_inputs

In [20]:
def main(optimizer='adam', loss='mean_squared_error', metrix='accuracy'):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = Conv2D(64, (4,4), padding="same", activation="relu")(cnn_inputs)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.2)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(32)(lstm_layers)
    lstm_layers = Dense(32, activation='relu')(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([layers, lstm_layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[cnn_inputs, lstm_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrix])
    return model

In [21]:
def main_2(optimizer='adam', loss='mean_squared_error', metrix='accuracy'):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = Conv2D(32, (3,3), padding="same", activation="relu")(cnn_inputs)
    layers = BatchNormalization()(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.3)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(16)(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([layers, lstm_layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[cnn_inputs, lstm_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrix])
    return model

In [29]:
def main_3(optimizer='adam', loss='mean_squared_error', metrix='accuracy'):
    #cnn
    cnn_inputs = Input(shape=(height, width, 1), name='cnn_inputs')
    layers = Conv2D(32, (3,3), padding="same", activation="relu")(cnn_inputs)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(32, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(64, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Conv2D(64, (3,3), activation='relu')(layers)
    layers = BatchNormalization()(layers)
    layers = MaxPooling2D(2,2, border_mode='same')(layers)
    layers = Flatten()(layers)
    layers = Dropout(0.3)(layers)
    layers = Dense(32, activation='relu')(layers)
    
    #lstm
    lstm_inputs = Input(shape=(steps, 24), name='lstm_inputs')
    lstm_layers = LSTM(16, return_sequences=True)(lstm_inputs)
    lstm_layers = LSTM(32)(lstm_layers)
    lstm_layers = Dense(32, activation='relu')(lstm_layers)
    lstm_layers = Dense(24, activation='relu')(lstm_layers)
    
    #main route
    main_ = concatenate([layers, lstm_layers])
    main_ = BatchNormalization()(main_)
    main_ = Dense(64, activation='relu')(main_)
    main_ = Dense(64, activation='relu')(main_)
    output = Dense(24, activation='sigmoid')(main_)
    
    model = Model(inputs=[cnn_inputs, lstm_inputs], outputs=[output])
    model.compile(optimizer=optimizer, loss=loss, metrics=[metrix])
    return model

In [22]:
def train_test_whole(classifier, num, name, epochs=100, batchsize=32):
    lstm_inputs, cnn_inputs, labels_inputs = inputs_to_model(train_order, num, 0, train_order[num].shape[0]+1)
    
    classifier.fit([cnn_inputs, lstm_inputs], labels_inputs, epochs=epochs, batch_size=batchsize, verbose=1,
                  validation_data = ([cnn_test, lstm_test], labels_test))
    
    #CV_loss, accuracy = load_test_score(classifier, cnn_test, lstm_test, y_test)
    #print('val_loss: ', CV_loss)
    #print('val_accuarcy', accuracy)
    
    classifier.save_weights(name, overwrite=True)   
    del classifier
    K.clear_session()

In [23]:
#Generate the CV sets
kf = KFold(n_splits=3, random_state=7)

train_order = []
test_order = []
length = np.arange(len(lstm))

for x,y in kf.split(length):
    train_order.append(length[x])
    test_order.append(length[y])
train_order = np.array(train_order)
test_order = np.array(test_order)

In [24]:
lstm_test, cnn_test, labels_test = inputs_to_model(test_order, 0, 0, test_order[0].shape[0]+1)

In [18]:
adam = keras.optimizers.Adam(lr=0.005)
classifier = main_2(optimizer=adam, loss='binary_crossentropy')
train_test_whole(classifier, 0, 'weight-32units-model2.hdf5', epochs=100)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  import sys


Train on 143394 samples, validate on 72556 samples
Epoch 1/100
143394/143394 [==============================] - 212s 1ms/step - loss: 0.0718 - acc: 0.9760 - val_loss: 0.0877 - val_acc: 0.9724
Epoch 2/100
143394/143394 [==============================] - 208s 1ms/step - loss: 0.0506 - acc: 0.9829 - val_loss: 0.0864 - val_acc: 0.9749
Epoch 3/100
143394/143394 [==============================] - 205s 1ms/step - loss: 0.0447 - acc: 0.9846 - val_loss: 0.0722 - val_acc: 0.9766
Epoch 4/100
143394/143394 [==============================] - 203s 1ms/step - loss: 0.0417 - acc: 0.9855 - val_loss: 0.0367 - val_acc: 0.9872
Epoch 5/100
143394/143394 [==============================] - 204s 1ms/step - loss: 0.0395 - acc: 0.9861 - val_loss: 0.0374 - val_acc: 0.9870
Epoch 6/100
143394/143394 [==============================] - 203s 1ms/step - loss: 0.0381 - acc: 0.9864 - val_loss: 0.0548 - val_acc: 0.9817
Epoch 7/100
143394/143394 [==============================] - 204s 1ms/step - loss: 0.0368 - acc: 0.9869

21312

In [27]:
adam = keras.optimizers.Adam(lr=0.01)
classifier = main_2(optimizer=adam, loss='binary_crossentropy')
train_test_whole(classifier, 0, 'weight-32units-model3.hdf5', epochs=100)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  import sys


Train on 139360 samples, validate on 70501 samples
Epoch 1/100
139360/139360 [==============================] - 181s 1ms/step - loss: 0.0552 - acc: 0.9812 - val_loss: 0.3805 - val_acc: 0.9392
Epoch 2/100
139360/139360 [==============================] - 180s 1ms/step - loss: 0.0356 - acc: 0.9882 - val_loss: 0.3828 - val_acc: 0.9467
Epoch 3/100
139360/139360 [==============================] - 180s 1ms/step - loss: 0.0312 - acc: 0.9898 - val_loss: 0.4036 - val_acc: 0.9478
Epoch 4/100
139360/139360 [==============================] - 180s 1ms/step - loss: 0.0291 - acc: 0.9906 - val_loss: 0.4484 - val_acc: 0.9432
Epoch 5/100
139360/139360 [==============================] - 180s 1ms/step - loss: 0.0273 - acc: 0.9913 - val_loss: 0.3851 - val_acc: 0.9490
Epoch 6/100
139360/139360 [==============================] - 182s 1ms/step - loss: 0.0257 - acc: 0.9917 - val_loss: 0.3981 - val_acc: 0.9509
Epoch 7/100
103264/139360 [=====================>........] - ETA: 42s - loss: 0.0247 - acc: 0.9921

KeyboardInterrupt: 

In [28]:
adam = keras.optimizers.Adam(lr=0.005)
classifier = main(optimizer=adam, loss='binary_crossentropy')
train_test_whole(classifier, 0, 'weight-32units-model3.hdf5', epochs=100)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  


Train on 139360 samples, validate on 70501 samples
Epoch 1/100
139360/139360 [==============================] - 185s 1ms/step - loss: 0.0650 - acc: 0.9778 - val_loss: 0.3571 - val_acc: 0.9479
Epoch 2/100
139360/139360 [==============================] - 183s 1ms/step - loss: 0.0418 - acc: 0.9863 - val_loss: 0.3671 - val_acc: 0.9478
Epoch 3/100
139360/139360 [==============================] - 184s 1ms/step - loss: 0.0376 - acc: 0.9877 - val_loss: 0.4646 - val_acc: 0.9326
Epoch 4/100
139360/139360 [==============================] - 183s 1ms/step - loss: 0.0354 - acc: 0.9885 - val_loss: 0.3690 - val_acc: 0.9536
Epoch 5/100
139360/139360 [==============================] - 181s 1ms/step - loss: 0.0339 - acc: 0.9889 - val_loss: 0.4094 - val_acc: 0.9449
Epoch 6/100
 39040/139360 [=======>......................] - ETA: 1:59 - loss: 0.0315 - acc: 0.9898

KeyboardInterrupt: 

In [30]:
adam = keras.optimizers.Adam(lr=0.005)
classifier = main_3(optimizer=adam, loss='binary_crossentropy')
train_test_whole(classifier, 0, 'weight-32units-model1.hdf5', epochs=100)
gc.collect()

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if __name__ == '__main__':
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  if sys.path[0] == '':
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(2, 2, padding="same")`
  from ipykernel import kernelapp as app


Train on 139360 samples, validate on 70501 samples
Epoch 1/100
139360/139360 [==============================] - 198s 1ms/step - loss: 0.0857 - acc: 0.9695 - val_loss: 0.3407 - val_acc: 0.9477
Epoch 2/100
139360/139360 [==============================] - 196s 1ms/step - loss: 0.0484 - acc: 0.9837 - val_loss: 0.3802 - val_acc: 0.9384
Epoch 3/100
 22784/139360 [===>..........................] - ETA: 2:26 - loss: 0.0418 - acc: 0.9862

KeyboardInterrupt: 